In [5]:
import polars as pl

# CVの計算を行う

In [6]:
cv_fold0 = pl.read_csv(
    "../"
    + "trained_models/e005-use-large-fold0_org/valid_dataset_e005-use-large-fold0.csv"
)
cv_fold1 = pl.read_csv(
    "../"
    + "trained_models/e006-use-large-fold1_org/valid_dataset_e006-use-large-fold1.csv"
)
cv_fold2 = pl.read_csv(
    "../"
    + "trained_models/e007-use-large-fold2_org/valid_dataset_e007-use-large-fold2.csv"
)

In [7]:
oof = pl.concat([cv_fold0, cv_fold1, cv_fold2])

In [8]:
oof

index,Clothing ID,Age,Title,Review Text,Rating,label,Positive Feedback Count,Division Name,Department Name,Class Name,fold,valid_pred
i64,i64,i64,str,str,i64,i64,i64,str,str,str,i64,f64
32,3,19,"""""","""This dress is so pretty!! the …",4,1,0,"""General""","""Dresses""","""Dresses""",0,0.996503
33,3,25,"""So beautiful!""","""This dress is just absolutely …",5,1,2,"""General""","""Dresses""","""Dresses""",0,0.9986541
34,3,26,"""Beautiful and unique dress""","""I purchased this dress for my …",5,1,0,"""General""","""Dresses""","""Dresses""",0,0.9985562
35,3,28,"""""","""I was looking for a dress for …",5,1,1,"""General""","""Dresses""","""Dresses""",0,0.9972638
36,3,31,"""High quality one of a kind pie…","""This dress had my attention im…",5,1,2,"""General""","""Dresses""","""Dresses""",0,0.9974788
…,…,…,…,…,…,…,…,…,…,…,…,…
9995,232,57,"""Runs big on top""","""""",3,1,5,"""General""","""Dresses""","""Dresses""",2,0.957912
9996,232,58,"""""","""I loved the dress, but just no…",1,1,5,"""General""","""Dresses""","""Dresses""",2,0.881258
9997,232,60,"""I was really disappointed""","""I was really hoping this dress…",2,0,7,"""General""","""Dresses""","""Dresses""",2,0.019308


In [9]:
oof.write_csv("../" + "trained_models/e005-6-7-ens/large-oof.csv")

In [10]:
from sklearn.metrics import roc_auc_score

cv_score = roc_auc_score(oof["label"], oof["valid_pred"])
print(f"CV Score: {cv_score}")

CV Score: 0.9693835695951963


In [11]:
pred_fold0 = pl.read_csv(
    "../"
    + "trained_models/e005-use-large-fold0_org/submission_e005-use-large-fold0_cv0.9719.csv"
)
pred_fold1 = pl.read_csv(
    "../"
    + "trained_models/e006-use-large-fold1_org/submission_e006-use-large-fold1_cv0.9737.csv"
)
pred_fold2 = pl.read_csv(
    "../"
    + "trained_models/e007-use-large-fold2_org/submission_e007-use-large-fold2_cv0.9654.csv"
)

In [13]:
from scipy.stats import rankdata

rankdata(pred_fold0)

array([ 7085.5,  1696. ,  8929. , ...,  9496.5,  8608. , 10664. ])

In [15]:
(rankdata(pred_fold0) + rankdata(pred_fold1) + rankdata(pred_fold2))

array([22882. ,  5281. , 27584. , ..., 27636. , 27141. , 29970.5])

In [23]:
# ((pred_fold0 + pred_fold1 + pred_fold2) / 3).write_csv(
sub = pl.DataFrame((rankdata(pred_fold0) + rankdata(pred_fold1) + rankdata(pred_fold2)))

sub.rename({"column_0": "target"}).write_csv(
    "../" + "trained_models/e005-6-7-ens/e005-6-7-ens-rank.csv"
)

target
f64
22882.0
5281.0
27584.0
2438.0
13478.5
…
27247.5
29776.0
27636.0
